# Day 20 - more path finding

This is essentially a path-finding problem; the regex is a tree of legal steps to reach a location, and from that tree you can construct all possible legal moves across the map. We can then find the longest possible path from this.

Note that the *shortest route* is required; no meandering around to get to the furstest room. The input probably includes ways to open doors between rooms that open up shortcuts to the furthest rooms. So we can build a graph of the rooms and connections first (taking care to *reuse* a room when it already exists at that location), then use a path finding algorithm to determine which path is longest. 

The 'regex' string should be treated as a stack; each time we reach a `(`, push our current state on the stack, and continue with the next character; each `|` is just another step back to the node at the top of the stack, and with `)` we can pop from the stack and continue where we left off.

Finding the longest path is a simple exhaustive traversal of the graph, where we avaid visiting rooms we have seen before unless our new path is shorter.

In [1]:
from collections import deque
from dataclasses import dataclass, field
from typing import Iterable, Iterator, Mapping, Optional, Tuple

Pos = Tuple[int, int]
_delta = {'N': (0, -1), 'W': (-1, 0), 'E': (1, 0), 'S': (0, 1)}
_doors = dict(zip('NWES', '-||-'))
_inverse = dict(zip('NWES', 'SEWN'))

# not really frozen, we allow manipulation through __getitem__ and __setitem__
# but the N, S, W, E directions are not part of the hash.
@dataclass(frozen=True)
class Room:
    x: int = 0
    y: int = 0
    N: 'Node' = field(default=None, compare=False, repr=False)
    W: 'Node' = field(default=None, compare=False, repr=False)
    S: 'Node' = field(default=None, compare=False, repr=False)
    E: 'Node' = field(default=None, compare=False, repr=False)

    @property
    def pos(self) -> Pos:
        return (self.x, self.y)

    def open_door(self, dir: str, map: Mapping[Pos, 'Room']) -> 'Room':
        room = self[dir]
        if room is None:
            dx, dy = _delta[dir]
            room = self[dir] = Room(self.x + dx, self.y + dy)
            room[_inverse[dir]] = self
        return room

    @property
    def available(self) -> Iterator[str]:
        return (d for d in 'NSWE' if self[d] is not None)
            
    def __getitem__(self, dir: str) -> 'Room':
        if dir not in 'NSWE':
            raise AttributeError(dir)
        return self.__dict__[dir]
    
    def __setitem__(self, dir: str, room: 'Room') -> None:
        if dir not in 'NSWE':
            raise AttributeError(dir)
        self.__dict__[dir] = room
    
    def __repr__(self) -> str:
        doors = ''.join(d if self[d] is not None else d.lower() for d in 'NSWE')
        return f"{type(self).__name__}(x={self.x}, y={self.y} {doors})"

    
class RegularMap:
    start: Optional[Room]

    def __init__(self, start: Room):
        self.start = start
        
    @classmethod
    def from_regex(cls, regex: Iterable[str]) -> 'RegularMap':
        start = here = Room()
        stack = deque()
        it = iter(regex)
        existing = {}
        assert next(it) == '^'
        for c in it:
            if c == '$':
                return RegularMap(start)
            elif c == '(':
                stack.append(here)
            elif c == '|':
                here = stack[-1]
            elif c == ')':
                here = stack.pop()
            else:
                here = here.open_door(c, existing)
    
    def furthest_room(self) -> int:
        queue = deque([(self.start, 0)])
        distances = {self.start.pos: 0}
        while queue:
            room, distance = queue.popleft()
            distance += 1
            for dir in room.available:
                next_ = room[dir]
                if next_.pos in distances and distances[next_.pos] <= distance:
                    continue
                distances[next_.pos] = distance
                queue.append((next_, distance))
        return max(distances.values())
    
    def __str__(self):
        queue = deque([self.start])
        # collect coords
        rooms = {self.start.pos: self.start}
        while queue:
            room = queue.popleft()
            for dir in room.available:
                next_ = room[dir]
                if next_.pos in rooms:
                    continue
                rooms[next_.pos] = next_
                queue.append(next_)
        minx, miny = (min(vs) for vs in zip(*rooms))
        maxx, maxy = (max(vs) for vs in zip(*rooms))
        width, height = abs(maxx + 1 - minx), abs(maxy + 1 - miny)
        lines = [['#'] * (width * 2 + 1) for _ in range(height * 2 + 1)]
        for (x, y), room in rooms.items():
            lines[(y - miny) * 2 + 1][(x - minx) * 2 + 1] = 'X' if room.pos == (0, 0) else '.' 
            for dir in room.available:
                dx, dy = _delta[dir]
                lines[(y - miny) * 2 + 1 + dy][(x - minx) * 2 + 1 + dx] = _doors[dir]
        
        return '\n'.join([''.join(l) for l in lines])

In [2]:
tests = {
    "^ENWWW(NEEE|SSE(EE|N))$": ('#########\n#.|.|.|.#\n#-#######\n#.|.|.|.#\n#-#####-#\n#.#.#X|.#\n#-#-#####\n#.|.|.|.#\n#########', None),
    "^ENNWSWW(NEWS|)SSSEEN(WNSE|)EE(SWEN|)NNN$": ('###########\n#.|.#.|.#.#\n#-###-#-#-#\n#.|.|.#.#.#\n#-#####-#-#\n#.#.#X|.#.#\n#-#-#####-#\n#.#.|.|.|.#\n#-###-###-#\n#.|.|.#.|.#\n###########', None),
    "^ESSWWN(E|NNENN(EESS(WNSE|)SSS|WWWSSSSE(SW|NNNE)))$": ('#############\n#.|.|.|.|.|.#\n#-#####-###-#\n#.#.|.#.#.#.#\n#-#-###-#-#-#\n#.#.#.|.#.|.#\n#-#-#-#####-#\n#.#.#.#X|.#.#\n#-#-#-###-#-#\n#.|.#.|.#.#.#\n###-#-###-#-#\n#.|.#.|.|.#.#\n#############', 23),
    "^WSSEESWWWNW(S|NENNEEEENN(ESSSSW(NWSW|SSEN)|WSWWN(E|WWS(E|SS))))$": ('###############\n#.|.|.|.#.|.|.#\n#-###-###-#-#-#\n#.|.#.|.|.#.#.#\n#-#########-#-#\n#.#.|.|.|.|.#.#\n#-#-#########-#\n#.#.#.|X#.|.#.#\n###-#-###-#-#-#\n#.|.#.#.|.#.|.#\n#-###-#####-###\n#.|.#.|.|.#.#.#\n#-#-#####-#-#-#\n#.#.|.|.|.#.|.#\n###############', 31),
}

for regex, (expected_str, expected_distance) in tests.items():
    test_rmap = RegularMap.from_regex(regex) 
    assert str(test_rmap) == expected_str
    if expected_distance:
        assert test_rmap.furthest_room() == expected_distance

In [3]:
import aocd

data = aocd.get_data(day=20, year=2018)
regular_map = RegularMap.from_regex(data)

In [4]:
print('Part 1:', regular_map.furthest_room())

Part 1: 3991
